# Segmenting and Clustering Neighborhoods in Toronto
***

First we will read in https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M as a `pandas` DataFrame using `read_html`

In [25]:
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# !conda install -c conda-forge geocoder 
from geopy.geocoders import Nominatim
import geocoder
# !conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import requests
import json

In [ ]:
url ="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df = pd.read_html(url)[0]
df.columns = ['Postcode', 'Borough', 'Neighborhood']
df = df.iloc[1:] # remove the first row since it was actually the column names
df = df[~df.Borough.str.contains("Not assigned")] # remove empty data
df = df.reset_index(drop=True) # reset the index for rows
df.head()

The next step is to combine Postcodes maintaining Borough names and concatenating Neighborhoods with a comma

In [3]:
df = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood]],Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [4]:
df.shape

(103, 3)

## Adding Latitude and Longitude for Boroughs
***

In [5]:
df["Latitude"] = ""
df["Longitude"] = ""
df.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",,
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",,
2,M1E,Scarborough,"Guildwood]],Morningside,West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,
5,M1J,Scarborough,Scarborough Village,,
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",,
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",,
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",,
9,M1N,Scarborough,"Birch Cliff,Cliffside West",,


In [6]:
for index, row in df.iterrows():
    # initialize your variable to None
    lat_lng_coords = None
    postal_code = row['Postcode']

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    row["Latitude"] = latitude
    row["Longitude"] = longitude

In [7]:
pd.set_option('precision', 8)
df.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood]],Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


Only keep Boroughs that contain "Toronto"

In [48]:
df = df[df.Borough.str.contains("Toronto")] # remove empty data
df = df.reset_index(drop=True) # reset the index for rows
df["Borough"].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [49]:
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.72802,-79.38879


In [50]:
df.shape

(38, 5)

## Generating a Map to Show Neighborhoods
***

## 1. Exploring the Dataset

Use geopy library to get the latitude and longitude values of Toronto.

In [51]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7067483, -79.59405439999999.


Create Map of Toronto

In [52]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Foursquare Credentials

In [53]:
CLIENT_ID = 'I1WYE1Z151WOQFKLAYX054S0RWGV50EHKBCQDTAIJIHIX2PQ' # your Foursquare ID
CLIENT_SECRET = 'O45ZLZKWMDTDAFIDZRKKOJFMLEXNCJOOUWIBQ3LDT0MSX33J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I1WYE1Z151WOQFKLAYX054S0RWGV50EHKBCQDTAIJIHIX2PQ
CLIENT_SECRET:O45ZLZKWMDTDAFIDZRKKOJFMLEXNCJOOUWIBQ3LDT0MSX33J


Exploring first neighborhood in dataframe

In [54]:
df.loc[0, 'Neighborhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [55]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.6763574, -79.2930312.


#### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [56]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=I1WYE1Z151WOQFKLAYX054S0RWGV50EHKBCQDTAIJIHIX2PQ&client_secret=O45ZLZKWMDTDAFIDZRKKOJFMLEXNCJOOUWIBQ3LDT0MSX33J&v=20180605&ll=43.6763574,-79.2930312&radius=500&limit=100'

In [57]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b9c0b476a607175a635bd5a'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e77e3861f6ecf8d3648300c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e0931735',
         'name': 'Coffee Shop',
         'pluralName': 'Coffee Shops',
         'primary': True,
         'shortName': 'Coffee Shop'}],
       'id': '4e77e3861f6ecf8d3648300c',
       'location': {'address': '637 Kingston Rd.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Main St.',
        'distance': 486,
        'formattedAddress': ['637 Kingston Rd. (at Main St.)',
         'Toronto ON M4E 1R3',
         'Canada'],
        'labeledLatLngs

In [58]:
# function that extracts the category of the venue
# Taken from the foursquare lab in coursera
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [59]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.67879837,-79.29804498
1,Grover Pub and Grub,Pub,43.67918143,-79.29721536
2,Upper Beaches,Neighborhood,43.68056321,-79.29286887
3,Beaches Fitness,Gym / Fitness Center,43.68031856,-79.29099083


In [60]:
print('{} venue(s) were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venue(s) were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [64]:
print(toronto_venues.shape)
toronto_venues.head()

(1690, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6763574,-79.2930312,Starbucks,43.67879837,-79.29804498,Coffee Shop
1,The Beaches,43.6763574,-79.2930312,Grover Pub and Grub,43.67918143,-79.29721536,Pub
2,The Beaches,43.6763574,-79.2930312,Upper Beaches,43.68056321,-79.29286887,Neighborhood
3,The Beaches,43.6763574,-79.2930312,Beaches Fitness,43.68031856,-79.29099083,Gym / Fitness Center
4,"The Danforth West,Riverdale",43.6795571,-79.3521880,Pantheon,43.67762124,-79.35143390,Greek Restaurant


Let's check how many venues were returned for each neighborhood

In [65]:
toronto_venues.groupby('Neighborhood').size().reset_index(name='counts')

,Neighborhood,counts
0,"Adelaide,King,Richmond",100
1,Berczy Park,55
2,"Brockton,Exhibition Place,Parkdale Village",21
3,Business reply mail Processing Centre969 Eastern,20
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",14
5,"Cabbagetown,St. James Town",47
6,Central Bay Street,85
7,"Chinatown,Grange Park,Kensington Market",100
8,Christie,16
9,Church and Wellesley,83


## 3. Analyze Each Neighborhood 

In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
toronto_onehot.shape

(1690, 233)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [70]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar
0,"Adelaide,King,Richmond",0.00,0.0,0.0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.01
1,Berczy Park,0.00,0.0,0.0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.00,0.0,0.0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
3,Business reply mail Processing Centre969 Eastern,0.05,0.0,0.0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.00,0.0,0.0,0.07142857,0.07142857,0.07142857,0.14285714,0.14285714,0.14285714,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
